### ベイジアンネットワーク
**ベイジアンネットワーク**は、グラフ表現によるネットワーク図をベースとした、変数間の関係性を表す手法のこと。  
**スケルトン**とは、骨格構造のことでDAGにおいて因果の方向を示す矢印がないグラフのこと。  
**PDAG**は、スケルトンとDAGの中間にあたる存在。  
  
ベイジアンネットワークでは、因果関係のあるノード間の関係性を**条件付き確率表(CPT: Conditional Probabilities Tables)**で表現する。

当てはまりの良さを示す指標としてここでは**BIC(Bayesian information criterion)**を使用する。
$$
BIC_m=-2l_m(\theta_m|\boldsymbol{X})+k_m(\log{N})
$$
ここで、$m$はモデル（対象のベイジアンネットワークのDAG）、$\boldsymbol{X}$は実測されたデータを示す。  
$k_m$はデータ数、$l_m(\theta_m|\boldsymbol{X})$はデータ$\boldsymbol{X}$のもとでのモデルの対数尤度を示す。

ベイジアンネットワークにおけるCPTを以下のように考える。  

|x1|P(x1)|
|---|---|
|0|0.6|
|1|0.4|

|x2|P(x2)|
|---|---|
|0|0.4|
|1|0.6|

|x1|x2|x3|P(x3=0\|x1,x2)|
|---|---|---|:---:|
|0|0|0|0.2|
|0|1|0|0.3|
|1|0|0|0.4|
|1|1|0|0.1|

パラメータ数は1+1+4=6となる。（2値変数なので、P(0)がわかれば、P(1)もわかる。）

In [1]:
import numpy as np
import pandas as pd

num_data = 10

x1 = np.random.choice([0, 1], num_data, p=[0.6, 0.4])
x2 = np.random.choice([0, 1], num_data, p=[0.4, 0.6])

df = pd.DataFrame({'x1': x1,
                   'x2': x2})
df.head()

,x1,x2
0,0,1
1,1,0
2,0,0
3,0,1
4,0,1


In [2]:
x3 = []
for i in range(num_data):
    if x1[i] == 0 and x2[i] == 0:
        x3_value = np.random.choice([0, 1], num_data, p=[0.2, 0.8])
        x3.append(x3_value[0])
    elif x1[i] == 0 and x2[i] == 1:
        x3_value = np.random.choice([0, 1], num_data, p=[0.3, 0.7])
        x3.append(x3_value[0])
    elif x1[i] == 1 and x2[i] == 0:
        x3_value = np.random.choice([0, 1], num_data, p=[0.4, 0.6])
        x3.append(x3_value[0])
    elif x1[i] == 1 and x2[i] == 1:
        x3_value = np.random.choice([0, 1], num_data, p=[0.1, 0.9])
        x3.append(x3_value[0])
df["x3"] = x3
df

,x1,x2,x3
0,0,1,0
1,1,0,1
2,0,0,1
3,0,1,1
4,0,1,0
5,0,1,0
6,1,0,0
7,1,0,0
8,0,0,1
9,0,1,1


CPTの具体的な確率値を$\hat{\theta}_{i,j,k}$と表す。  
$$
\hat{\theta}_{i,j,k}=\frac{N_{ijk}}{N_{ij}}
$$
ここで$N_{ijk}$は変数$i$がとある条件パターン$j$で値$k$となったデータ数。  
$N_{ij}$は変数$i$がとある条件パターン$j$であったデータ数。 
  
変数$x_1$は条件付き確率ではないので、条件パターン$j$は存在しない。  
よって、$\hat{\theta}_{1,[\,],0}=4/10=0.4$となる。  
また、変数$x_3$は条件パターン$j$として、\[0,0\]、\[0,1\]、\[1,0\]、\[1,1\]の4つとなる。  
例えば、$\hat{\theta}_{3,[0,1],0}=1/3=0.33$と計算できる。  
  
対数尤度$l_m(\theta_m|\boldsymbol{X})$は計算が大変なため、比例する値を計算する。  
$$
l_m(\theta_m|\boldsymbol{X}) \propto \sum_i\sum_j\sum_k(N_{ijk})\log{\hat{\theta}_{i,j,k}}
$$

In [6]:
from pgmpy.models import BayesianModel
model = BayesianModel([('x1', 'x3'), ('x2', 'x3')])

In [8]:
from pgmpy.estimators import ParameterEstimator
pe = ParameterEstimator(model, df)
print("\n", pe.state_counts('x1'))
print("\n", pe.state_counts('x2'))
print("\n", pe.state_counts('x3'))


    x1
0   7
1   3

    x2
0   5
1   5

 x1    0         1     
x2    0    1    0    1
x3                    
0   0.0  3.0  2.0  0.0
1   2.0  2.0  1.0  0.0


In [9]:
from pgmpy.estimators import BayesianEstimator

estimator = BayesianEstimator(model, df)

cpd_x1 = estimator.estimate_cpd(
    'x1', prior_type="dirichlet", pseudo_counts=[[0],[0]])
cpd_x2 = estimator.estimate_cpd(
    'x2', prior_type="dirichlet", pseudo_counts=[[0],[0]])
cpd_x3 = estimator.estimate_cpd(
    'x3', prior_type="dirichlet", pseudo_counts=[[0,0,0,0],[0,0,0,0]])

print(cpd_x1)
print(cpd_x2)
print(cpd_x3)

+-------+-----+
| x1(0) | 0.7 |
+-------+-----+
| x1(1) | 0.3 |
+-------+-----+
+-------+-----+
| x2(0) | 0.5 |
+-------+-----+
| x2(1) | 0.5 |
+-------+-----+
+-------+-------+-------+--------------------+-------+
| x1    | x1(0) | x1(0) | x1(1)              | x1(1) |
+-------+-------+-------+--------------------+-------+
| x2    | x2(0) | x2(1) | x2(0)              | x2(1) |
+-------+-------+-------+--------------------+-------+
| x3(0) | 0.0   | 0.6   | 0.6666666666666666 | nan   |
+-------+-------+-------+--------------------+-------+
| x3(1) | 1.0   | 0.4   | 0.3333333333333333 | nan   |
+-------+-------+-------+--------------------+-------+


C:\Users\tanak\anaconda3\envs\tanaka_env1\lib\site-packages\pgmpy\factors\discrete\CPD.py:285: RuntimeWarning: invalid value encountered in true_divide
  tabular_cpd.values = (cpd / cpd.sum(axis=0)).reshape(tabular_cpd.cardinality)


In [10]:
from pgmpy.estimators import BicScore
bic = BicScore(df)
print(bic.score(model))

-25.22247094506125


### 独立性の検定
#### カイ二乗検定

In [15]:
num_data = 100

x1 = np.random.choice([0, 1], num_data, p=[0.6, 0.4])
x2 = np.random.choice([0, 1], num_data, p=[0.4, 0.6])
x2 = x2*x1

df = pd.DataFrame({'x1':x1,
                   'x2':x2})
df.head()

,x1,x2
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [16]:
print(((df["x1"]==0)&(df["x2"]==0)).sum())
print(((df["x1"]==1)&(df["x2"]==0)).sum())
print(((df["x1"]==0)&(df["x2"]==1)).sum())
print(((df["x1"]==1)&(df["x2"]==1)).sum())

59
17
0
24


カイ二乗統計量は、
$$
\chi^2=\sum_{i=1}^{r}\sum_{j=1}^{c}\frac{(n_{ij}-E_{ij})^2}{E_{ij}}
$$
$E_{ij}$は推定期待度数であり、
$$
E_{ij}=\frac{n_jn_j}{N}
$$

In [18]:
num_data = 100

# 独立である状況
x1 = np.random.choice([0, 1], num_data, p=[0.6, 0.4])
x2 = np.random.choice([0, 1], num_data, p=[0.4, 0.6])

df2 = pd.DataFrame({'x1':x1,
                   'x2':x2})
df2.head()

,x1,x2
0,0,1
1,0,0
2,1,1
3,0,1
4,1,1


In [19]:
print(((df2["x1"]==0)&(df2["x2"]==0)).sum())
print(((df2["x1"]==1)&(df2["x2"]==0)).sum())
print(((df2["x1"]==0)&(df2["x2"]==1)).sum())
print(((df2["x1"]==1)&(df2["x2"]==1)).sum())

17
22
38
23


In [53]:
from pgmpy.estimators import ConstraintBasedEstimator

est = ConstraintBasedEstimator.ConstraintBasedEstimator(df2)
print(est.test_conditional_independence('x1', 'x2', method="chi_square", tol=0.05))

est = ConstraintBasedEstimator(df)
print(est.test_conditional_independence('x1', 'x2', method="chi_square", tol=0.05))

AttributeError: 'ConstraintBasedEstimator' object has no attribute 'test_conditional_independence'

### PCアルゴリズムによるベイジアンネットワーク探索

In [31]:
from scipy.special import expit

num_data = 2000

x = np.random.uniform(low=-1, high=1, size=num_data)

e_z = np.random.randn(num_data)
z_prob = expit(-5.0*x+5*e_z)
Z = np.array([])

for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)
    
t = np.zeros(num_data)
for i in range(num_data):
    if x[i] < 0:
        t[i] = 0.5
    elif x[i] >= 0 and x[i] < 0.5:
        t[i] = 0.7
    elif x[i] >= 0.5:
        t[i] = 1.0
        
e_y = np.random.randn(num_data)
Y = 2.0 + t*Z + 0.3*x + 0.1*e_y

In [34]:
Y2 = np.random.choice([1.0, 2.0, 3.0, 4.0, 5.0], num_data, p=[0.1, 0.2, 0.3, 0.2, 0.2])

e_y3 = np.random.randn(num_data)
Y3 = 3*Y + Y2 + e_y3

e_y4 = np.random.randn(num_data)
Y4 = 3*Y3 + 5 + 2*e_y4

In [35]:
df = pd.DataFrame({'x':x,
                   'Z':Z,
                   't':t,
                   'Y':Y,
                   'Y2':Y2,
                   'Y3':Y3,
                   'Y4':Y4})
df.head()

,x,Z,t,Y,Y2,Y3,Y4
0,0.430652,1.0,0.7,2.860354,1.0,9.222925,30.867631
1,-0.193179,1.0,0.5,2.409125,3.0,8.908337,32.392158
2,-0.175358,1.0,0.5,2.437063,3.0,9.314384,31.661338
3,-0.161702,1.0,0.5,2.550986,4.0,13.026968,43.373926
4,0.805733,0.0,1.0,2.325156,5.0,12.093357,43.432567


In [36]:
s_qcut, bins = pd.cut(df["Y"], 5, labels=[1, 2, 3, 4, 5], retbins=True) # ビンごとにデータ数が同じになる

print(s_qcut)
print(bins)

0       4
1       3
2       3
3       3
4       3
       ..
1995    2
1996    2
1997    1
1998    2
1999    1
Name: Y, Length: 2000, dtype: category
Categories (5, int64): [1 < 2 < 3 < 4 < 5]
[1.55595831 1.94081874 2.32376443 2.70671013 3.08965583 3.47260153]


In [37]:
df_bin = df.copy()
del df_bin["t"]

df_bin["x"], x_bins = pd.cut(df["x"], 5, labels=[1, 2, 3, 4, 5], retbins=True)
df_bin["Z"], x_bins = pd.cut(df["Z"], 5, labels=[1, 2, 3, 4, 5], retbins=True)
df_bin["Y"], x_bins = pd.cut(df["Y"], 5, labels=[1, 2, 3, 4, 5], retbins=True)
df_bin["Y2"], x_bins = pd.cut(df["Y2"], 5, labels=[1, 2, 3, 4, 5], retbins=True)
df_bin["Y3"], x_bins = pd.cut(df["Y3"], 5, labels=[1, 2, 3, 4, 5], retbins=True)
df_bin["Y4"], x_bins = pd.cut(df["Y4"], 5, labels=[1, 2, 3, 4, 5], retbins=True)

df_bin.head()

,x,Z,Y,Y2,Y3,Y4
0,4,5,4,1,2,2
1,3,5,3,3,2,2
2,3,5,3,3,2,2
3,3,5,3,4,4,4
4,5,1,3,5,4,4


In [52]:
from pgmpy.estimators import ConstraintBasedEstimator
import pgmpy

est = ConstraintBasedEstimator.ConstraintBasedEstimator(df_bin)

print(est.test_conditional_independence('x', 'Z', method="chi_square", tol=0.05))

AttributeError: 'ConstraintBasedEstimator' object has no attribute 'test_conditional_independence'